In [ ]:
base_path = '/home/PERSONALE/nicolas.biondini2/Interactomes/Sliding_hole_removal'

d_dist_df_list = [[]]*len(os.listdir(base_path))
i = 0

for directory in os.listdir(base_path):
    
    path = os.path.join(base_path, directory)
    
    for file in os.listdir(path):
        if file.startswith('removal'):
            df_path = os.path.join(path, file)
        elif file.startswith('genes'):
            nodes_list_path = os.path.join(path, file)


    nodes_df = pd.read_csv(nodes_list_path)
    d_dist_df_list[i] = pd.read_csv(df_path)    
    d_dist_df_list[i]['nodes'] = pd.Series(dtype="object")
    
    # Crea una mappatura da Posizione a Gene
    mapping = dict(zip(nodes_df['Posizione'], nodes_df['Gene']))
    # Usa la mappatura per assegnare la colonna 'Nodes_Removed'
    d_dist_df_list[i]['nodes'] = d_dist_df_list[i].index.map(mapping)

    i += 1


#creo il DF con le aree
d_dist_areas_df = pd.DataFrame(index = range (len(os.listdir (base_path) )), columns = ['Area', 'Nodes_removed', 'Position'] )
i = 0

for df in d_dist_df_list:
    d_dist_areas_df.at[i, 'Nodes_removed'] = df['nodes'].dropna().tolist()
    d_dist_areas_df.at[i, 'Position'] = int(df[df['nodes'].notna()].index.tolist()[0])
    d_dist_areas_df.at[i, 'Area'] = integrate.simpson(df['S'] , df['f'])
    i += 1

#calcolo la resilienza
d_dist_areas_df['Resilience'] = 0.5 - d_dist_areas_df['Area']

In [ ]:
### Convert Ensembl IDs to HGNC symbols using mygene.info and Ensembl REST API

import gseapy as gp
import mygene
import requests
import time

def convert_ensembl_to_hgnc(ensembl_ids, verbose=False):
    """
    Convert a list of Ensembl gene IDs to HGNC symbols using mygene.info and Ensembl REST API.
    Args:
        ensembl_ids (list): List of Ensembl gene IDs to convert.
        verbose (bool): If True, print progress messages.
    Returns:
        dict: A dictionary mapping Ensembl IDs to HGNC symbols.
    """
    mg = mygene.MyGeneInfo()
    converted = {}

    # Primo tentativo: mygene.info
    if verbose: print("🔍 Provo con mygene.info...")
    results = mg.querymany(
        ensembl_ids,
        scopes='ensembl.gene',
        fields='symbol',
        species='human',
        as_dataframe=False
    )

    not_found = []

    for r in results:
        ensg = r['query']
        if 'symbol' in r:
            converted[ensg] = r['symbol']
        else:
            not_found.append(ensg)
            if verbose: print(f"❌ Non trovato in mygene: {ensg}")

    # Secondo tentativo: Ensembl REST API
    if verbose: print("🔄 Provo con Ensembl REST API per quelli mancanti...")
    for ensg in not_found:
        url = f"https://rest.ensembl.org/xrefs/id/{ensg}?content-type=application/json;external_db=HGNC"
        r = requests.get(url, headers={"Content-Type": "application/json"})
        if r.ok:
            data = r.json()
            if data:
                symbol = data[0]['display_id']
                converted[ensg] = symbol
                if verbose: print(f"✅ Trovato via Ensembl: {ensg} → {symbol}")
            else:
                if verbose: print(f"❌ Ancora non trovato: {ensg}")
        else:
            if verbose: print(f"⚠️ Errore con Ensembl API per {ensg}: {r.status_code}")
        time.sleep(0.1)  # rate limiting

    return converted


In [ ]:
### Enrichment Analysis ###


def gene_enrichment_analysis(gene_list, gene_set=['GO_Biological_Process_2025'], organism='Human'):
    """
    Esegue l'analisi di arricchimento dei geni utilizzando Enrichr.
    Args:
        gene_list (list): Lista di geni da analizzare.
    Returns:
        pd.DataFrame: Risultati dell'analisi di arricchimento.
    """

    import gseapy as gp

    enr = gp.enrichr(
        gene_list=gene_list,
        gene_sets = gene_set,
        organism = organism,
        outdir=None,  # Non salvare su disco
        cutoff=0.05
    )
    return enr.results


In [ ]:
## TROVO IL VALORE DI TAGLIO PER d_dist_areas_hugo_df

# Estrai i dati di resilience
resilience_data = d_dist_areas_hugo_df['Resilience'].dropna().astype(float).values

# Calcola l'istogramma con 1000 bin
counts, bin_edges = np.histogram(resilience_data, bins=1000)

# Trova i bin che corrispondono all'intervallo richiesto
mask_bins = (bin_edges[:-1] >= 0.45225) & (bin_edges[1:] <= 0.45233) #intervallo scelto "a vista" dal grafico. Il punto di minimo è lì dentro

# Seleziona i conteggi e i centri dei bin nell'intervallo
selected_counts = counts[mask_bins]
selected_bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
selected_bin_centers = selected_bin_centers[mask_bins]

# Trova il valore di resilience meno popoloso nell'intervallo
if len(selected_counts) > 0:
    idx_min = np.argmin(selected_counts)
    min_resilience = selected_bin_centers[idx_min]
    min_count = selected_counts[idx_min]
    print(f"Valore di resilience meno popoloso: {min_resilience}, conteggio: {min_count}")
else:
    print("Nessun bin trovato nell'intervallo specificato.")


cut_value = 0.4523077770321452 #valore trovato con il metodo sopra

#seleziono i geni con resilience maggiore del cut_value
mask = (d_dist_areas_df['Resilience'].astype(float) >= cut_value)   

genes_of_interest = d_dist_areas_df[mask].dropna()['Nodes_removed'].explode().dropna().unique().tolist()


# Converti gli Ensembl IDs in HGNC symbols
gene_symbols = list( convert_ensembl_to_hgnc(genes_of_interest, verbose=True).values() )

# Esegui l'analisi di arricchimento
peak_enr = gene_enrichment_analysis(gene_symbols, gene_set=['GO_Biological_Process_2025'], organism='Human')

#save the significant results
peak_enr.loc[peak_enr['Adjusted P-value'] < 0.05].sort_values(by='Adjusted P-value').to_csv('db2020_peak_enrichment.csv', index=False)